# Week 2 Submission - Children's Story Generator

Generates educational short stories for children designed to teach specified values and good behavior.

NOTE:
- The system uses an OpenAI model when an OpenAI API key is available; otherwise, it falls back to Ollama.

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

can_use_openai = True if api_key else False

In [ ]:
openai = OpenAI()
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

system_prompt = """
  You are an experienced motherly children story teller who is great at telling children's story that are short, inspiring and amazing for children.",

  Guidelines:
  - Give short stories that exemplify good behavior that a good parent will want to teach their child
  - Give short stories that exemplify good behavior such as kindness, listening to parents, respect, honesty etc
  - Avoid stories that are difficult for kids to understand
  - Respond in markdown format
"""

def create_user_prompt(specific_value = "Kindness"):
  return f"""
  Generate a story for children that can be shared within 5 minutes before bedtime

  Remember to:
  - Give short stories that exemplify good behavior and specifically about {specific_value}
  - Avoid stories that are difficult for kids to understand
  - Respond in markdown format
"""

In [ ]:
GPT_MODEL = 'gpt-4o-mini'
OLLAMA_MODEL = 'llama3.2'

In [ ]:
def reader(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",
      input=message
    )
    return response.content

In [ ]:
def generate_story_stream(story_topic):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": create_user_prompt(story_topic)}
    ]

    if can_use_openai:
        response = openai.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )
    else:
        response = ollama.chat.completions.create(
            model=OLLAMA_MODEL,
            messages=messages,
        )

    story = response.choices[0].message.content
    voice = reader(story)

    return story, voice

In [ ]:
with gr.Blocks(
    title="Children's Story Generator",
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="blue"
    )
) as view:

    gr.Markdown("## 📚 Children's Story Generator")

    message_input = gr.Textbox(
        label="What should the story be about?",
        placeholder="Enter a topic...",
        lines=2
    )

    generate_btn = gr.Button("Generate Story")

    message_output = gr.Textbox(
        label="Your Story😀😀😀",
        lines=12
    )

    audio_output = gr.Audio(
        label="Listen to the Story",
        type="filepath"
    )

    gr.Examples(
        examples=[
            ["Kindness"],
            ["Listening to parents"],
            ["Respect"],
            ["Honesty"],
            ["Obedience"],
        ],
        inputs=message_input,
    )

    generate_btn.click(
        fn=generate_story_stream,
        inputs=message_input,
        outputs=[message_output, audio_output],
    )

view.launch()